In [2]:
import scipy.integrate as integrate
import numpy as np
pi = np.pi

Added lambda, fixed integral, P cannot be all same otherwise singular matrix

  
Need to work on the other two terms of W.

In [18]:
num_GTO = int(input('how many GTOs?'))

#add GTOs to chi
str_chi = 'd[0]*GTO_1(r,a[0])'

for i in range(2, num_GTO +1):
    str_chi += '+ d[{0}]*GTO_1(r,a[{0}]) '.format(i-1)
    
#str_chi += '\n'

# initial value of P
#P = np.zeros(2*num_GTO + 1) + 0.1
P = np.array([i*0.001 for i in range (1,2*num_GTO + 2)])

'''
#a array
a = P[0 : num_GTO]
#d list
d = P[num_GTO : 2*num_GTO]

lambd = P[2*num_GTO]
'''
GTO_1 = lambda r,a: (2*a/pi)**(3/4)*np.exp(-a*r**2)
STO_1 = lambda r: pi**(-1/2)*np.exp(-r)

#par_GTO_a = lambda r,i: (GTO_1(r,a[i-1]))
#par_GTO_d = lambda r,i: (GTO_1(r,a[i-1]))*(3/(4*a[i-1] - r**2))

def par_GTO(r,i):
    if i <= num_GTO:
        #a_i derivative
        return d[i-1]*(3/(4*a[i-1]) - r**2)*GTO_1(r,a[i-1])
    if i == 2*num_GTO + 1:
        return 0
    elif i > num_GTO:
        #b_i derivative
        return GTO_1(r,a[i-1-num_GTO])

def double_par_GTO(r,j,k):
    if j==k:
        if j <= num_GTO:
            return d[j-1]*((3/(4*a[j-1]) - r**2)**2 - 3/(4*a[j-1]**2))*GTO_1(r,a[j-1])
        if j > num_GTO:
            return 0
    if max(j,k) > num_GTO and min (j,k) <= num_GTO:
        #a_i and d_i derivative
        i = min(j,k)
        return (3/(4*a[i-1-num_GTO]) - r**2)*(GTO_1(r,a[i-1-num_GTO]))
    else:
        return 0

def integ(x):
    if x == 0:
        print('here')
        return 0
    else:
        return integrate.quad(lambda r: x(r)*r**2 ,0, np.inf)[0]

how many GTOs? 2


In [20]:
#a array
a = P[0 : num_GTO]
#d list
d = P[num_GTO : 2*num_GTO]

lambd = P[2*num_GTO]

main = '''
#All integral definitions
B_integral = lambda i : -integ(lambda r:(STO_1(r) - (1-lambd)*{0})*par_GTO(r,i))

lambd_integral = lambda i : -integ(lambda r:{0}*par_GTO(r,i))

W_integral = lambda j,k : (1-lambd)*(integ(lambda r: par_GTO(r,j)*par_GTO(r,k))\
+ integ(lambda r: {0}*double_par_GTO(r,j,k)))\
- integ(lambda r: (STO_1(r)*double_par_GTO(r,j,k)))

#B
list_B = [B_integral(i) for i in range(1,2*num_GTO + 1)] + \
[0.5*(1 - integ(lambda r: ({0}**2 )))]
array_B = np.array(list_B)

#for our lagrange multiplier
last_row = [lambd_integral(k) for k in range (1,2*num_GTO+1)]\
+[0]

'''.format(str_chi)

exec(main)

# list_W contains many lists, which are rows each
list_W = [[W_integral(j,k) for k in range (1,2*num_GTO+1)]\
          + [lambd_integral(j)] for j in range (1,2*num_GTO+1)]\
            + [last_row]

array_W = np.array(list_W)

solution = np.linalg.solve(array_W, array_B)
P += solution

print(P)

<ipython-input-18-4f8a737d17af>:23: RuntimeWarning: invalid value encountered in double_scalars
  GTO_1 = lambda r,a: (2*a/pi)**(3/4)*np.exp(-a*r**2)
<ipython-input-18-4f8a737d17af>:23: RuntimeWarning: overflow encountered in exp
  GTO_1 = lambda r,a: (2*a/pi)**(3/4)*np.exp(-a*r**2)
<ipython-input-18-4f8a737d17af>:57: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(lambda r: x(r)*r**2 ,0, np.inf)[0]


[nan nan nan nan nan]


In [7]:
W_integral(1,2)

<ipython-input-3-3caf6e82ea36>:49: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(lambda r: x(r)*r**2 ,0, np.inf)[0]


nan

In [9]:
r=1
double_par_GTO(r,1,2)

0